In [ ]:
from langserve.client import RemoteRunnable

rag_app = RemoteRunnable("http://127.0.0.1:8008/chat")
rag_app.invoke("How much wifes can i have?")

In [ ]:
from rag_local.prompts import MuslimImamPrompt

prompt = MuslimImamPrompt().build()
print(prompt.pretty_repr())

In [ ]:

from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.embeddings import GPT4AllEmbeddings
from langchain_community.embeddings.huggingface import (
    HuggingFaceEmbeddings,
    HuggingFaceBgeEmbeddings,
)
from rag_local.embeddings import HuggingFaceBgeEmbeddingsComponent

model_name = "BAAI/bge-small-en-v1.5"
model_kwargs  = {"device": "cpu", "trust_remote_code": True}
embeddings = HuggingFaceBgeEmbeddingsComponent().build(
            model_name=model_name,
            model_kwargs=model_kwargs,
        )

loader = TextLoader("quran_test.md")
documents = loader.load()[:4]

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
docs = text_splitter.split_documents(documents)
# embeddings = OpenAIEmbeddings()
db = FAISS.from_documents(docs, embeddings)
print(db.index.ntotal)

In [ ]:
query = "What did the president say about Ketanji Brown Jackson"
docs = db.similarity_search(query)
# docs = db.similarity_search_with_score(query)
docs

In [30]:
from rag_local.prompts import MuslimImamPrompt, QuestionAnswerPrompt
from rag_local.chain_custom import ChainBuilder

prompt = QuestionAnswerPrompt().build()


In [31]:
def retrieve_docs(query: str):
    pairs = db.similarity_search_with_score(query)
    docs, scores = zip(*pairs)
    return pairs

In [32]:
import json


json.dumps(retrieve_docs(query), ensure_ascii=False)

Batches: 100%|██████████| 1/1 [00:00<00:00, 30.30it/s]


[(Document(page_content='# 1707', metadata={'source': 'quran_test.md'}),
  0.8419801),
 (Document(page_content='# 666', metadata={'source': 'quran_test.md'}),
  0.87291646),
 (Document(page_content='# 4208', metadata={'source': 'quran_test.md'}),
  0.8837631),
 (Document(page_content='# 1708', metadata={'source': 'quran_test.md'}),
  0.8892559)]

In [ ]:
from pprint import pprint
from langchain_core.runnables import (
    RunnableParallel,
    RunnablePassthrough,
    RunnableLambda,
)

from rag_local.utils import first

prompt = QuestionAnswerPrompt().build()
docs_with_scores = RunnableLambda(retrieve_docs)
rag_chain = (
        RunnableParallel({"context": docs_with_scores, "question": RunnablePassthrough()})
        | prompt
    )

pprint(rag_chain.invoke("How much wifes can i have?"))